In [73]:
import requests
import sqlite3
import os
from operator import itemgetter

GRADE_POINTS = {
    "A+": 4.0, "A": 4.0, "A-": 3.7, 
    "B+": 3.3, "B": 3.0, "B-": 2.7,
    "C+": 2.3, "C": 2.0, "C-": 1.7,
    "D+": 1.3, "D": 1.0, "D-": 0.5,
    "F": 0.0
}
        
def db_ensure_tables_exist(conn):
    # courses stores the overall info for a course
    conn.cursor().execute("""
    CREATE TABLE IF NOT EXISTS courses (
        id CHAR(8) NOT NULL PRIMARY KEY,
        dept CHAR(4) NOT NULL,
        number INT(4) NOT NULL,
        credits INT(1) NOT NULL
    )
    """)
    # course_professors stores the professors for a course
    conn.cursor().execute("""
    CREATE TABLE IF NOT EXISTS course_professors (
        course CHAR(8) NOT NULL,
        name VARCHAR(100) NOT NULL,
        CONSTRAINT primary_key PRIMARY KEY (course, name),
        FOREIGN KEY (course) REFERENCES courses(id)
    )
    """)
    # You cannot create a primary key containing a TEXT field, 
    # so we don't have one for this table.
    conn.cursor().execute("""
    CREATE TABLE IF NOT EXISTS professor_reviews (
        course CHAR(8) NOT NULL,
        prof_name VARCHAR(100) NOT NULL,
        rating INT(1) NOT NULL,
        review TEXT NOT NULL,
        FOREIGN KEY (course) REFERENCES courses(id),
        FOREIGN KEY (prof_name) REFERENCES course_professors(name)
    )
    """)
    # course_grades stores the gpa and number of drops for a course
    # in a given year and semester. the semester column is either
    # "FALL" or "SPRING".
    conn.cursor().execute("""
    CREATE TABLE IF NOT EXISTS course_grades (
        course CHAR(8) NOT NULL,
        prof_name VARCHAR(100),
        year INT(4) NOT NULL,
        semester VARCHAR(10) NOT NULL,
        gpa DOUBLE(2,2) NOT NULL,
        num_drops INT(5) NOT NULL,
        CONSTRAINT primary_key PRIMARY KEY (course, prof_name, year, semester),
        FOREIGN KEY (course) REFERENCES courses(id),
        FOREIGN KEY (prof_name) REFERENCES course_professors(name)
    )
    """)

def db_setup(wipe=False):
    file_name = "project.db"
    if wipe:
        os.remove(file_name)
    conn = sqlite3.connect(file_name)
    db_ensure_tables_exist(conn)
    return conn

def db_write_course_info(conn, plt_terp_info):
    dept, number, credits, prof_names, reviews = itemgetter(
        "department",
        "course_number", 
        "credits",
        "professors",
        "reviews"
    )(plt_terp_info)
    cur = conn.cursor()

    id = f"{dept}{number}"
    cur.execute(
        "INSERT INTO courses (id, dept, number, credits) VALUES (?, ?, ?, ?)", 
        (id, dept, number, credits)
    )
    
    for prof_name in prof_names:
        print(f"prof {prof_name} for course id {id}")
        cur.execute(
            "INSERT OR IGNORE INTO course_professors (course, name) VALUES (?, ?)",
            (id, prof_name)
        )
    
    for review in reviews:
        # perhaps do sentiment analysis here on the review text?
        cur.execute(
            "INSERT INTO professor_reviews (course, prof_name, rating, review) VALUES (?, ?, ?, ?)",
            (id, review["professor"], review["rating"], review["review"])
        )

def db_write_course_grades(conn, course_id, plt_terp_grades):
    cur = conn.cursor()
    for entry in plt_terp_grades:
        num_grade_w = int(entry["W"])
        prof_name = entry["professor"]
        semester_raw = entry["semester"]
        year = semester_raw[0:4]
        semester_id = int(semester_raw[4:])
        semester_enum = "FALL" if semester_id == 1 else "SPRING" if semester_id == 8 else None
            
        # if we can't identify fall or spring semester,
        # then the course isnt relevant to our data
        if semester_enum == None:
            continue
            
        # loop over keys and values and check if key is a grade
        # name. if so, add it to our grade point sum and total
        # amount of grades.
        grade_point_sum = 0
        total_grades = 0
        for key, value in entry.items():
            if key not in GRADE_POINTS:
                continue
            
            grade_points = GRADE_POINTS[key]
            amt = int(value)
            total_grades += amt
            grade_point_sum += amt * grade_points
        
        gpa = grade_point_sum / total_grades
        print(f"gpa of {gpa} for semester {semester_enum} of {year} for course {course_id}")
        cur.execute(
            """
            INSERT INTO course_grades (course, prof_name, year, semester, gpa, num_drops)
            VALUES (?, ?, ?, ?, ?, ?)
            ON CONFLICT(course, prof_name, year, semester) 
                DO UPDATE SET gpa = (gpa + excluded.gpa) / 2 AND num_drops = num_drops + excluded.num_drops
            """,
            (course_id, prof_name, year, semester_enum, gpa, num_grade_w)
        )

def get_cmsc_courses():
    result = requests.get("https://api.umd.io/v1/courses", params={"dept_id": "CMSC"})
    return list(map(lambda x: x["course_id"], result.json()))

def get_course_info(course_id):
    return requests.get("https://api.planetterp.com/v1/course", params={"name": course_id, "reviews": "true"}).json()

def get_course_grades(course_id):
    return requests.get("https://api.planetterp.com/v1/grades", params={"course": course_id}).json()
    
with db_setup(wipe=True) as conn:
    for course_id in get_cmsc_courses():
        db_write_course_info(conn, get_course_info(course_id))
        db_write_course_grades(conn, course_id, get_course_grades(course_id))
        print(f"processed {course_id}")

prof Charles Kassir for course id CMSC100
prof Amy Vaillancourt for course id CMSC100
prof Alyssa Neuner for course id CMSC100
prof Andrew Nolan for course id CMSC100
prof Savannah Renehan for course id CMSC100
prof Jessica Long for course id CMSC100
prof Corie Brown for course id CMSC100
prof Veronica Sanchez for course id CMSC100
gpa of 3.96875 for semester SPRING of 2021 for course CMSC100
gpa of 3.805882352941176 for semester SPRING of 2021 for course CMSC100
gpa of 3.8947368421052633 for semester SPRING of 2021 for course CMSC100
gpa of 3.9 for semester SPRING of 2018 for course CMSC100
gpa of 3.9375 for semester SPRING of 2018 for course CMSC100
gpa of 3.7588235294117647 for semester SPRING of 2017 for course CMSC100
gpa of 3.96 for semester SPRING of 2017 for course CMSC100
gpa of 3.3315789473684214 for semester SPRING of 2016 for course CMSC100
gpa of 3.382608695652174 for semester SPRING of 2015 for course CMSC100
gpa of 3.6694444444444443 for semester SPRING of 2014 for cours